In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%cd MyDrive/'ML Project '/Data/query
!ls

Mounted at /gdrive
/gdrive
/gdrive/MyDrive/ML Project /Data/query
full_labeled_data.csv  outvectors.npy  train1.json  train2.json
outlabels.npy	       train1.csv      train2.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [3]:
data = np.load("outvectors.npy")
labels = np.load("outlabels.npy")

data.shape, labels.shape

((55002, 300), (55002,))

In [6]:
per = np.random.permutation(len(data))

data = data[per]
labels = labels[per]


In [7]:
x_train = data[:50000]
y_train = labels[:50000]

x_test = data[50000:]
y_test = labels[50000:]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 300), (50000,), (5002, 300), (5002,))

In [10]:
(unique, counts) = np.unique(y_train, return_counts=True)
least = min(counts)

counts, least

(array([ 8010, 33930,  8060]), 8010)

In [11]:
pos = np.asarray(data[np.nonzero(y_train == 1)])
neut = np.asarray(data[np.nonzero(y_train == 0)])
neg = np.asarray(data[np.nonzero(y_train == -1)])
pos.shape, neg.shape, neut.shape

((8060, 300), (8010, 300), (33930, 300))

In [12]:
x_train_bal = np.concatenate((pos[:least], neg[:least], neut[:least]))
y_train_bal = np.concatenate((np.ones((least,)), -1 * np.ones((least,)), np.zeros((least,))))

x_train_bal.shape, y_train_bal.shape

((24030, 300), (24030,))

In [13]:
perm = np.random.permutation(len(x_train_bal))

x_train_bal = x_train_bal[perm]
y_train_bal = y_train_bal[perm]



In [14]:
(unique, counts) = np.unique(y_test, return_counts=True)
counts

array([ 810, 3431,  761])

CLASSIFY USING BALANCED DATA

In [15]:
clf = svm.SVC()
clf.fit(x_train_bal, y_train_bal)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
y_pred = clf.predict(x_test)
f1 = f1_score(y_pred, y_test, average = None)
recall = recall_score(y_pred, y_test, average = None)
f1, recall

(array([0.58987459, 0.80138112, 0.67834935]),
 array([0.47282204, 0.91927575, 0.5952381 ]))

TRAIN ON ORIGINAL DATA

In [17]:
clf2 = svm.SVC()
clf2.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [18]:
y_pred2 = clf2.predict(x_test)
f1 = f1_score(y_pred2, y_test, average = None)
recall = recall_score(y_pred2, y_test, average = None)
f1, recall

(array([0.55309396, 0.87840785, 0.69904341]),
 array([0.7254509 , 0.82509603, 0.79431438]))

In [19]:
(unique, counts) = np.unique(y_pred2, return_counts=True)
least = min(counts)

counts, least

(array([ 499, 3905,  598]), 499)

MAKE 2 CLASSIFIERS, NEUT vs ALL, then POS vs Neg

In [20]:
x_train_pn = np.concatenate((pos,neg))
y_train_pn = np.concatenate((np.ones(len(pos),), -1 * np.ones((len(neg),))))

x_train_neut = np.concatenate((pos,neg,neut))
y_train_neut = np.concatenate((np.ones((len(pos) + len(neg),)), np.zeros((len(neut),))))

perm1 = np.random.permutation(len(y_train_pn))
x_train_pn = x_train_pn[perm1]
y_train_pn = y_train_pn[perm1] 

perm2 = np.random.permutation(len(y_train_neut))

x_train_neut = x_train_neut[perm2]
y_train_neut = y_train_neut[perm2]

x_train_pn.shape, y_train_pn.shape, x_train_neut.shape, y_train_neut.shape

((16070, 300), (16070,), (50000, 300), (50000,))

In [21]:
clf_neut = svm.SVC()
clf_pn = svm.SVC()
clf_neut.fit(x_train_neut, y_train_neut)
clf_pn.fit(x_train_pn, y_train_pn)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [24]:
y_pred = np.zeros((len(y_test),))

pn_pred = clf_pn.predict(x_test)
neut_pred = clf_neut.predict(x_test)

for i in range(len(neut_pred)):
  if neut_pred[i] != 0:
    y_pred[i] = pn_pred[i]

In [25]:
f1 = f1_score(y_pred, y_test, average = None)
recall = recall_score(y_pred, y_test, average = None)
f1, recall

(array([0.55259467, 0.87540204, 0.70497547]),
 array([0.63961039, 0.84139785, 0.75525526]))